In [2]:
# 使用迁移学习的思想，以VGG16作为模板搭建模型，训练识别手写字体
# 引入VGG16模块
from keras.applications.vgg16 import VGG16
from keras.layers import Input
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model
from keras.optimizers import SGD

# 加载字体库作为训练样本
from keras.datasets import mnist

# 加载OpenCV（在命令行中窗口中输入pip install opencv-python），这里为了后期对图像的处理，
# 大家使用pip install C:\Users\28542\Downloads\opencv_python-3.4.1+contrib-cp35-cp35m-win_amd64.whl
# 比如尺寸变化和Channel变化。这些变化是为了使图像满足VGG16所需要的输入格式
import cv2
import h5py as h5py
import numpy as np

In [3]:
# 建立一个模型，其类型是Keras的Model类对象，我们构建的模型会将VGG16顶层去掉，只保留其余的网络
# 结构。这里用include_top = False表明我们迁移除顶层(顶层及vgg的最后一层)以外的其余网络结构到自己的模型中
# VGG模型对于输入图像数据要求高宽至少为48个像素点，由于硬件配置限制，我们选用48个像素点而不是原来
# VGG16所采用的224个像素点。即使这样仍然需要24GB以上的内存，或者使用数据生成器
model_vgg = VGG16(include_top=False, weights='imagenet', input_shape=(48, 48, 3))
for layer in model_vgg.layers:
    layer.trainable = False # 设置不可以训练了，也就是参数不动
model = Flatten(name='flatten')(model_vgg.output)
model = Dense(4096, activation='relu', name='fc1')(model)
model = Dense(4096, activation='relu', name='fc2')(model)
model = Dropout(0.5)(model)
model = Dense(10, activation='softmax')(model)
model_vgg_mnist = Model(inputs=model_vgg.input, outputs=model, name='vgg16')

# 打印模型结构，包括所需要的参数
model_vgg_mnist.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [48]:
sgd = SGD(lr=0.05, decay=1e-5) # decay学习率的衰减参数
model_vgg_mnist.compile(loss='categorical_crossentropy',
                                 optimizer=sgd, metrics=['accuracy'])

# 因为VGG16对网络输入层的要求，我们用OpenCV把图像从32*32变成224*224，把黑白图像转成RGB图像
# 并把训练数据转化成张量形式，供keras输入
def load_data(path="./test_data/mnist.npz"):
    f = np.load(path)
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']
    f.close()
    return (x_train, y_train), (x_test, y_test)

In [49]:
(X_train, y_train), (X_test, y_test) = load_data()

X_train, y_train = X_train[:10000], y_train[:10000]
X_test, y_test = X_test[:1000], y_test[:1000]

In [50]:
X_train.shape, y_train.shape

((10000, 28, 28), (10000,))

In [51]:
X_train = [cv2.cvtColor(cv2.resize(i, (48, 48)), cv2.COLOR_GRAY2RGB)
           for i in X_train]
X_test = [cv2.cvtColor(cv2.resize(i, (48, 48)), cv2.COLOR_GRAY2RGB)
          for i in X_test]

In [52]:
np.array(X_train).shape, np.array(y_train).shape

((10000, 48, 48, 3), (10000,))

In [53]:
X_train[0].shape

(48, 48, 3)

In [54]:
X_train[0][np.newaxis].shape

(1, 48, 48, 3)

In [55]:
## np.newaxis:增加一个维度
X_train = np.concatenate([arr[np.newaxis] for arr in X_train]).astype('float32')

X_test = np.concatenate([arr[np.newaxis] for arr in X_test]).astype('float32')

In [56]:
X_train.shape

(10000, 48, 48, 3)

In [57]:
print(X_train.shape)
print(X_test.shape)

X_train = X_train / 255
X_test = X_test / 255


def tran_y(y):
    y_ohe = np.zeros(10)
    y_ohe[y] = 1
    return y_ohe


y_train_ohe = np.array([tran_y(y_train[i]) for i in range(len(y_train))])
y_test_ohe = np.array([tran_y(y_test[i]) for i in range(len(y_test))])

model_vgg_mnist.fit(X_train, y_train_ohe, validation_data=(X_test, y_test_ohe),
                             epochs=100, batch_size=128,verbose=1)

(10000, 48, 48, 3)
(1000, 48, 48, 3)
Train on 10000 samples, validate on 1000 samples
Epoch 1/100
10000/10000 [==============================] - 12s 1ms/step - loss: 1.3963 - acc: 0.5651 - val_loss: 1.4003 - val_acc: 0.4190
Epoch 2/100
10000/10000 [==============================] - 9s 853us/step - loss: 0.7563 - acc: 0.7653 - val_loss: 0.7394 - val_acc: 0.7520
Epoch 3/100
10000/10000 [==============================] - 9s 856us/step - loss: 0.5668 - acc: 0.8197 - val_loss: 1.3852 - val_acc: 0.5420
Epoch 4/100
10000/10000 [==============================] - 9s 850us/step - loss: 0.5087 - acc: 0.8348 - val_loss: 1.2686 - val_acc: 0.5560
Epoch 5/100
10000/10000 [==============================] - 9s 872us/step - loss: 0.4188 - acc: 0.8694 - val_loss: 0.3756 - val_acc: 0.8890
Epoch 6/100
10000/10000 [==============================] - 9s 855us/step - loss: 0.4252 - acc: 0.8652 - val_loss: 0.6861 - val_acc: 0.7660
Epoch 7/100
10000/10000 [==============================] - 9s 870us/step - loss: 

Epoch 58/100
10000/10000 [==============================] - 9s 862us/step - loss: 0.0922 - acc: 0.9711 - val_loss: 0.2664 - val_acc: 0.9190
Epoch 59/100
10000/10000 [==============================] - 9s 865us/step - loss: 0.0984 - acc: 0.9687 - val_loss: 0.9320 - val_acc: 0.7250
Epoch 60/100
10000/10000 [==============================] - 9s 862us/step - loss: 0.1260 - acc: 0.9643 - val_loss: 0.6343 - val_acc: 0.8080
Epoch 61/100
10000/10000 [==============================] - 9s 862us/step - loss: 0.1437 - acc: 0.9587 - val_loss: 0.1410 - val_acc: 0.9520
Epoch 62/100
10000/10000 [==============================] - 9s 858us/step - loss: 0.0870 - acc: 0.9717 - val_loss: 0.1379 - val_acc: 0.9590
Epoch 63/100
10000/10000 [==============================] - 9s 860us/step - loss: 0.0817 - acc: 0.9745 - val_loss: 0.9795 - val_acc: 0.7600
Epoch 64/100
10000/10000 [==============================] - 9s 861us/step - loss: 0.1123 - acc: 0.9673 - val_loss: 0.3600 - val_acc: 0.8880
Epoch 65/100
10000/1